In [1]:
import pandas as pd

In [2]:
import geopandas as gp

In [3]:
import numpy as np

In [4]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
# from lxml import etree
from copy import copy
import os
import inspect
from xml.dom import minidom

In [5]:
import math

In [6]:
import networkx as nx

In [7]:
from networkx import DiGraph

In [8]:
from vrpy import VehicleRoutingProblem

In [9]:
import matplotlib.pyplot as plt 

In [10]:
# TODO: Not handling heterogeneous fleet right now
# TODO: Handle case where you have heterogeneous fleet, where one fleet part of the fleet can handle one commodity
# and the other a different commodity id.

In [11]:
# How does time window constraint relate cost on graph edges
# Time in mock up files in minutes
# can demand be pick-up by multiple vehicles?

### Function to create graph for carrier tour problem

In [142]:
# The first step is to create a graph for each carrier depending on its delivery/pickup location and depo locations
# This function receives as input, a data frame with payload for the carrier of interest, a data frame on the carrier
# of interest, the depot location, 
def create_graph(df_prob, carrier_id, c_prob, depot_loc, fleet_type,taz_df, tt_df):
    # Create an empty Directed graph and add the depot as source and sink to the digraph
    g = DiGraph()
    payload_nodes = dict()    # nodes associate with each payload id
    distance = dict()
    node_list = []
    missing_taz = False
    
    # Adding Nodes
    n = 1   # Node ids
    for i in df_prob['payload_id'].unique(): # Can we assume that payload id's are unique?
        if str(df_prob.loc[df_prob['payload_id'] == i]['job'].values[0]) != 'pickup_delivery':
            if str(df_prob.loc[df_prob['payload_id'] == i]['job'].values[0]) == 'delivery':
                temp_zone = (int(df_prob.loc[df_prob['payload_id'] == i]['del_zone'].values[0]))
            elif str(df_prob.loc[df_prob['payload_id'] == i]['job'].values[0]) =='pickup':
                temp_zone = (int(df_prob.loc[df_prob['payload_id'] == i]['pu_zone'].values[0]))
                   
            # determine taz corresponding to zone
            try:
                taz_id = int(taz_df[taz_df['cbg_ids'].str.contains('-'+ str(temp_zone)+ '-', na=False) |
                                   taz_df['cbg_ids'].str.contains('-'+ str(temp_zone)+ ',', na=False) |
                                   taz_df['cbg_ids'].str.contains(','+ str(temp_zone)+ '-', na=False) |
                                   taz_df['cbg_ids'].str.contains(','+ str(temp_zone)+ ',', na=False)]['taz1454'].values[0])
            except Exception as e:
                missing_taz = True
                print("Could not find taz for zone: ", temp_zone)
                break
                
#             # We are adding cbg_ids as nodes and not taz ids anymore
            g.add_node(taz_id)
#             pos[n] = copy(temp_pos)
            payload_nodes[i] = taz_id
#             print("i is ", i)
            n = n + 1

        else:
            temp_del = int(df_prob.loc[df_prob['payload_id'] == i]['del_zone'].values[0])
            temp_pick = int(df_prob.loc[df_prob['payload_id'] == i]['pu_zone'].values[0])
            
            try:
                taz_del = int(taz_df[taz_df['cbg_ids'].str.contains('-'+ str(temp_del)+ '-', na=False) |
                                   taz_df['cbg_ids'].str.contains('-'+ str(temp_del)+ ',', na=False) |
                                   taz_df['cbg_ids'].str.contains(','+ str(temp_del)+ '-', na=False) |
                                   taz_df['cbg_ids'].str.contains(','+ str(temp_del)+ ',', na=False)]['taz1454'].values[0])
                taz_pick = int(taz_df[taz_df['cbg_ids'].str.contains('-'+ str(temp_pick)+ '-', na=False) |
                               taz_df['cbg_ids'].str.contains('-'+ str(temp_pick)+ ',', na=False) |
                               taz_df['cbg_ids'].str.contains(','+ str(temp_pick)+ '-', na=False) |
                               taz_df['cbg_ids'].str.contains(','+ str(temp_pick)+ ',', na=False)]['taz1454'].values[0])
#                 print(taz_del, " to ", taz_pick)
            except Exception as e:
                missing_taz = True
                print("Could not find taz for zones: ", temp_del, " or ", temp_pick)
                break
            
            # Adding pick-up node
            g.add_node(taz_pick)
            payload_nodes[i] = [taz_pick]
            n = n + 1
            
            # Adding drop-off node    
            g.add_node(taz_del)
            payload_nodes[i].append(taz_del)
            n = n + 1
    
    try:
        depot_taz = taz_df[taz_df['cbg_ids'].str.contains(str(depot_loc), na=False)]['taz1454'].values[0]
    except Exception as e:
        missing_taz = True
        print("Could not find taz for depot: ", depot_loc)
                
    # Now completing the creation of the graph
    # Do below only if there are no missing data
    if not missing_taz:
        if any(isinstance(i, list) for i in list(payload_nodes.values())):
            node_list = [item for sublist in list(payload_nodes.values()) for item in sublist]
        else:
            node_list = list(payload_nodes.values())

#         print("node values: ", node_list)
        g = nx.complete_graph(node_list, create_using=None)
        g = g.to_directed()
        
        # Adding source and sink
        g.add_node("Source")
        # Sink location is same as depo 
        g.add_node("Sink")

        # Adding edge costs
        if fleet_type =='hete':
            print('hetegeneous fleet')
            # Determine number of vehicle types
            num_veh_types = sum('num_veh_type_' in s for s in c_prob.columns)
            print('number of vehicle types = ', num_veh_types)
            for edge in g.edges():
                startnode=edge[0] 
                endnode=edge[1]
                try:
                    dist_cost = float(tt_df.loc[(tt_df['origTaz'] == startnode) & (tt_df['destTaz'] == endnode)
                          & (tt_df['mode'] == 'CAR')]['travelTimeInS'].values[0])/60.  # Converting to minutes
                except Exception as e:
                    dist_cost = 3*60.0   # We assume 3 hours if we do not have travel time

                # We assume same travel time for medium and heavy duty vehicles, this can change in future
                # We assume medium and heavy duty vehicles for now
                g[startnode][endnode]['cost']= [dist_cost, dist_cost] # This should be probably be related to time window
                distance[(startnode,endnode)] = dist_cost
#                 print("I get here, dist_cost ", dist_cost)

        else:
            print('homogeneous fleet')
            for edge in g.edges():
                startnode=edge[0] 
                endnode=edge[1]
                try:
                    dist_cost = float(tt_df.loc[(tt_df['origTaz'] == startnode) & (tt_df['destTaz'] == endnode)
                          & (tt_df['mode'] == 'CAR')]['travelTimeInS'].values[0])/60.0  # converting to minutes
                except Exception as e:
                    dist_cost = 3*60.0   # We assume 3 hours if we do not have travel time
                g[startnode][endnode]["cost"]= dist_cost # This should be probably be related to time window
                distance[(startnode,endnode)] = dist_cost
                print("I get here, dist_cost: ", dist_cost)

        for n in node_list:  # Can we assume that payload id's are unique?
            # Add edge to source and sink
            try:
                dist_source = float(tt_df.loc[(tt_df['origTaz'] == depot_taz) & (tt_df['destTaz'] == n)
                          & (tt_df['mode'] == 'CAR')]['travelTimeInS'].values[0])/60.0
            except Exception as e:
                print("could not find travel time")
                dist_source = 3*60.0

            try:
                dist_sink = float(tt_df.loc[(tt_df['origTaz'] == n) & (tt_df['destTaz'] == depot_taz)
                          & (tt_df['mode'] == 'CAR')]['travelTimeInS'].values[0])/60.0
            except Exception as e:
                print("could not find travel time")
                dist_sink = 3*60.0

            if fleet_type == 'hete':
    #             print("dist_sink ", dist_sink, " dist_source ", dist_source)
                g.add_edge(n, "Sink", cost=[dist_sink, dist_sink])
                g.add_edge("Source", n, cost=[dist_source, dist_source])
            else:
                g.add_edge(n, "Sink", cost= dist_sink)
                g.add_edge("Source", n, cost= dist_source)

            distance[('Source',n)] = dist_source
            distance[(n,'Sink')] = dist_sink
    
    return g, payload_nodes, distance, missing_taz

### Function to add demand to problem

In [13]:
# Function to add the demand on the network graph for the problem
def add_demand(df_prob, g, payload_nodes):
    prob_type = ''
    for i in df_prob['payload_id'].unique(): 
        if str(df_prob.loc[df_prob['payload_id'] == i]['job'].values[0]) != 'pickup_delivery':
            # Add demand for delivery
            g.nodes[payload_nodes[i]]["demand"] = int(df_prob.loc[df_prob['payload_id'] == i]['weight'].values[0])
            
            # Adding stop duration and time window for delivery and pickup
            if str(df_prob.loc[df_prob['payload_id'] == i]['job'].values[0]) == 'delivery':
                # Adding time window and service time if they exist
                g.nodes[payload_nodes[i]]["lower"] = float(df_prob.loc[df_prob['payload_id'] == i]['del_tw_lower'].values[0])
                g.nodes[payload_nodes[i]]["upper"] = float(df_prob.loc[df_prob['payload_id'] == i]['del_tw_upper'].values[0])
                g.nodes[payload_nodes[i]]["service_time"] = float(df_prob.loc[df_prob['payload_id'] == i]['del_stop_duration'].values[0])
                prob_type = 'pickup'
            elif str(df_prob.loc[df_prob['payload_id '] == i]['job'].values[0]) =='pickup':
                g.nodes[payload_nodes[i]]["lower"] = float(df_prob.loc[df_prob['payload_id '] == i]['pu_tw_lower'].values[0])
                g.nodes[payload_nodes[i]]["upper"] = float(df_prob.loc[df_prob['payload_id '] == i]['pu_tw_upper'].values[0])
                g.nodes[payload_nodes[i]]["service_time"] = float(df_prob.loc[df_prob['payload_id '] == i]['pu_stop_duration'].values[0])
                prob_type = 'delivery'
        else:
            # TO DO this need to be fixed/looked at: what if you have multible delivery/pickup in same location
            # So far what I have seen is that a node with multiple demand is divided into multiple locations
            #Adding pick up demand
            pick_node = int(payload_nodes[i][0])
            del_node = int(payload_nodes[i][1])
            g.nodes[pick_node]["request"] = del_node
            demand = int(df_prob.loc[df_prob['payload_id'] == i]['weight'].values[0]);
            g.nodes[pick_node]["demand"] = demand
            # It is a pick-delivery process
            g.nodes[pick_node]["lower"] = float(df_prob.loc[df_prob['payload_id'] == i]['pu_tw_lower'].values[0])
            g.nodes[pick_node]["upper"] = float(df_prob.loc[df_prob['payload_id'] == i]['pu_tw_upper'].values[0])
            g.nodes[pick_node]["service_time"] = float(df_prob.loc[df_prob['payload_id'] == i]['pu_stop_duration'].values[0])
            
            # Adding delivery demand
            g.nodes[del_node]["demand"] = -demand
            # It is a pick-delivery process
            g.nodes[del_node]["lower"] = float(df_prob.loc[df_prob['payload_id'] == i]['del_tw_lower'].values[0])
            g.nodes[del_node]["upper"] = float(df_prob.loc[df_prob['payload_id'] == i]['del_tw_upper'].values[0])
            g.nodes[del_node]["service_time"] = float(df_prob.loc[df_prob['payload_id'] == i]['del_stop_duration'].values[0])
            prob_type = 'pickup_delivery'
            
    return prob_type

### Running the VRP Problems

In [14]:
taz_df = gp.read_file('input_file/TAZs/tazs_with_ids.shp')

In [15]:
taz_df.head()

,objectid,taz1454,district,county,gacres,Shape__Are,Shape__Len,cbg_ids,geometry
0,1,566,11,Santa Clara,1708.683056,0.000444,0.097550,3183,"POLYGON ((-121.86806 37.28720, -121.86888 37.2..."
1,2,565,11,Santa Clara,486.200669,0.000126,0.054171,"3184,3185","POLYGON ((-121.85877 37.28024, -121.85757 37.2..."
2,3,573,11,Santa Clara,538.069933,0.000140,0.057053,"3202,3203","POLYGON ((-121.81238 37.28889, -121.81298 37.2..."
3,4,571,11,Santa Clara,450.407906,0.000117,0.052042,"3204,3205","POLYGON ((-121.80597 37.27730, -121.80847 37.2..."
4,5,576,11,Santa Clara,423.273023,0.000110,0.043438,"3208,3209","POLYGON ((-121.82788 37.30855, -121.82848 37.3..."


In [16]:
taz_df['cbg_ids'] = '-' + taz_df['cbg_ids'] + '-'

In [17]:
taz_df.head()

,objectid,taz1454,district,county,gacres,Shape__Are,Shape__Len,cbg_ids,geometry
0,1,566,11,Santa Clara,1708.683056,0.000444,0.097550,-3183-,"POLYGON ((-121.86806 37.28720, -121.86888 37.2..."
1,2,565,11,Santa Clara,486.200669,0.000126,0.054171,"-3184,3185-","POLYGON ((-121.85877 37.28024, -121.85757 37.2..."
2,3,573,11,Santa Clara,538.069933,0.000140,0.057053,"-3202,3203-","POLYGON ((-121.81238 37.28889, -121.81298 37.2..."
3,4,571,11,Santa Clara,450.407906,0.000117,0.052042,"-3204,3205-","POLYGON ((-121.80597 37.27730, -121.80847 37.2..."
4,5,576,11,Santa Clara,423.273023,0.000110,0.043438,"-3208,3209-","POLYGON ((-121.82788 37.30855, -121.82848 37.3..."


In [21]:
len(taz_df)

1454

In [22]:
# taz_df[taz_df['taz1454']==565]['cbg_ids'].values

In [23]:
# We need to know the depot using the carrier file
c_df = pd.read_csv('input_file/carriers.csv')
c_df = c_df.dropna(axis=1, how='all')   # Removing all nan

# reading payload definition
p_df = pd.read_csv('input_file/payloads.csv')
p_df = p_df.dropna(axis=1, how='all')   # Removing all nan

In [24]:
c_df.head()

,carrier_id,firm_id,depot_zone,contract_firms,num_veh_type_1,num_veh_type_2
0,47,47,3504,[47],5.0,7.0
1,382,382,4374,[382],2.0,9.0
2,383,383,2569,[383],5.0,12.0
3,384,384,1091,[384],4.0,8.0
4,386,386,3882,[386],0.0,13.0


In [25]:
p_df['carrier_id'] = p_df['carrier_id'].astype(int)

In [26]:
p_df['sequence_id'] = np.nan

In [27]:
p_df['tour_id'] = np.nan

In [28]:
p_df['pu_arrival_time'] = np.nan

In [29]:
p_df['del_arrival_time'] = np.nan

In [76]:
p_df = p_df.fillna(int(0));

In [77]:
p_df.head()

,payload_id,carrier_id,commodity,weight,job,pu_zone,del_zone,pu_stop_duration,del_stop_duration,pu_tw_lower,pu_tw_upper,del_tw_lower,del_tw_upper,sequence_id,tour_id,pu_arrival_time,del_arrival_time
0,0,47,1,6145,delivery,0.0,310,0.0,7,0.0,0.0,300,1200,0.0,1.0,0.0,0.0
1,1,382,1,13574,delivery,0.0,316,0.0,18,0.0,0.0,300,1200,0.0,0.0,0.0,0.0
2,2,383,1,6888,delivery,0.0,300,0.0,5,0.0,0.0,300,1200,0.0,0.0,0.0,0.0
3,3,384,1,13574,delivery,0.0,316,0.0,30,0.0,0.0,300,1200,0.0,0.0,0.0,0.0
4,4,386,1,5358,delivery,0.0,316,0.0,14,0.0,0.0,300,1200,0.0,0.0,0.0,0.0


In [78]:
# Changing tour id and sequence id into ints
p_df['tour_id'] = p_df['tour_id'].astype(int)
p_df['sequence_id'] = p_df['sequence_id'].astype(int)

In [79]:
p_df.head()

,payload_id,carrier_id,commodity,weight,job,pu_zone,del_zone,pu_stop_duration,del_stop_duration,pu_tw_lower,pu_tw_upper,del_tw_lower,del_tw_upper,sequence_id,tour_id,pu_arrival_time,del_arrival_time
0,0,47,1,6145,delivery,0.0,310,0.0,7,0.0,0.0,300,1200,0,1,0.0,0.0
1,1,382,1,13574,delivery,0.0,316,0.0,18,0.0,0.0,300,1200,0,0,0.0,0.0
2,2,383,1,6888,delivery,0.0,300,0.0,5,0.0,0.0,300,1200,0,0,0.0,0.0
3,3,384,1,13574,delivery,0.0,316,0.0,30,0.0,0.0,300,1200,0,0,0.0,0.0
4,4,386,1,5358,delivery,0.0,316,0.0,14,0.0,0.0,300,1200,0,0,0.0,0.0


In [33]:
# Reading in vehicle information
v_df = pd.read_csv('input_file/vehicle_types.csv')
v_df = v_df.dropna(axis=1, how='all')   # Removing all nan

In [34]:
v_df.head()

,veh_type_id,veh_category,commodities,weight,payload_capacity_weight
0,1,MD,"[1, 2, 3, 4]",10000,5000
1,2,HD,"[1, 2, 3, 4]",20000,15000


In [35]:
vc_df = pd.read_csv('input_file/vehicles_by_carrier.csv')

In [36]:
vc_df.head()

,Unnamed: 0,carrier_id,md_veh,hd_veh,md_start_id,hd_start_id
0,0,45,3,9,0,3
1,1,46,3,10,12,15
2,2,47,5,7,25,30
3,3,48,3,9,37,40
4,4,49,9,8,49,58


In [37]:
vc_df.columns

Index(['Unnamed: 0', 'carrier_id', 'md_veh', 'hd_veh', 'md_start_id',
       'hd_start_id'],
      dtype='object')

In [38]:
tt_df = pd.read_csv('input_file/15.skims.csv')

In [39]:
tt_df.head()

,hour,mode,origTaz,destTaz,travelTimeInS,generalizedTimeInS,cost,generalizedCost,distanceInM,numObservations,energy
0,9,WALK_TRANSIT,127,101,519.0,668.0,1.950000,11.273257,3315.748870,1,2.957121e+04
1,7,CAR,286,542,2642.0,2642.0,5.729773,37.333762,52241.103000,1,8.934316e+07
2,1,CAR,27,946,256.0,253.0,0.671969,1.498766,5414.071000,1,9.689059e+06
3,4,DRIVE_TRANSIT,1196,22,4734.0,5288.0,7.424777,28.306041,63734.329526,1,1.644712e+07
4,15,CAR,1263,1240,973.0,3239.8,2.777605,38.819498,24848.288000,1,4.477590e+07


In [41]:
tt_df['mode'].unique()

array(['WALK_TRANSIT', 'CAR', 'DRIVE_TRANSIT', 'WALK', 'RIDE_HAIL',
       'BIKE', 'RIDE_HAIL_POOLED', 'RIDE_HAIL_TRANSIT'], dtype=object)

In [42]:
c_df = c_df.fillna(0); # Fill all nan with zeros

In [43]:
p_df['carrier_id'].unique()

array([     47,     382,     383, ..., 2248045, 2235463, 2189004])

In [44]:
g = DiGraph()

In [60]:
p_df.head()

,payload_id,carrier_id,commodity,weight,job,pu_zone,del_zone,pu_stop_duration,del_stop_duration,pu_tw_lower,pu_tw_upper,del_tw_lower,del_tw_upper
0,0,47,1,6145,delivery,0.0,310,0.0,7,0.0,0.0,300,1200
1,1,382,1,13574,delivery,0.0,316,0.0,18,0.0,0.0,300,1200
2,2,383,1,6888,delivery,0.0,300,0.0,5,0.0,0.0,300,1200
3,3,384,1,13574,delivery,0.0,316,0.0,30,0.0,0.0,300,1200
4,4,386,1,5358,delivery,0.0,316,0.0,14,0.0,0.0,300,1200


In [46]:
c_df.head()

,carrier_id,firm_id,depot_zone,contract_firms,num_veh_type_1,num_veh_type_2
0,47,47,3504,[47],5.0,7.0
1,382,382,4374,[382],2.0,9.0
2,383,383,2569,[383],5.0,12.0
3,384,384,1091,[384],4.0,8.0
4,386,386,3882,[386],0.0,13.0


### Creating the output dataframe to save as csv

In [146]:
tour_id = 0

In [147]:
tour_df = pd.DataFrame(columns = ['tour_id', 'veh_id', 'start_time', 'start_zone'])

In [148]:
for carr_id in c_df['carrier_id'].unique(): #p_df['carrier_id'].unique():
# for carr_id in [2187753]:
    
    #TO DO: when origin and destination zone fall in the same TAZ zone
    
    # Otherwise we use the depot_zone for location
    depot_loc = c_df.loc[c_df['carrier_id'] == carr_id]['depot_zone'].values[0]
    
    # To simplify the problem, look at a small problem with same carrier and same commodity id
    df_prob = p_df[(p_df['carrier_id'] == carr_id)]
    
    if len(df_prob) > 0:    
        missing_taz = True

        # Drop nan's columns
        df_prob = df_prob.dropna(axis=1, how='all')

        # Extract the data for the carrier we are interested in
        f_prob = vc_df[vc_df['carrier_id'] == carr_id]

        c_prob = c_df[c_df['carrier_id'] == carr_id]

        # Check if the carrier has homogeneous fleet to formulate a homogeneous fleet problem
        # This happens if the carrier only has medium duty vehicles or heavy duty vehicles
    #     if len(f_prob['veh_type_id'].unique()) == 1:
        if(f_prob['md_veh'].values[0] == 0 or f_prob['hd_veh'].values[0] == 0):
            print("\nNew Problem: Solving Homogeneous Problem for Carrier ", carr_id)
            fleet_type = 'homo'     

            # We assume first value in graph is medium duty and second is duty
            if f_prob['md_veh'].values[0] == 0:
                veh_capacity = int(v_df[v_df['veh_category'] == 'HD']['payload_capacity_weight'].values[0])
                num_veh = f_prob['hd_veh'].values[0]
            else:
                veh_capacity = int(v_df[v_df['veh_category'] == 'MD']['payload_capacity_weight'].values[0])
                num_veh = f_prob['md_veh'].values[0]

    #         print("veh_capacity: ", veh_capacity, " num_veh: ", num_veh)
            fleet_mix = False

        else:
            print("\n New Problem: Solving problem with heterogeneous fleet for carrier ", carr_id)
            fleet_type = 'hete'

            # We assume first value in graph is medium duty and second is duty
            veh_capacity = [int(v_df[v_df['veh_category'] == 'MD']['payload_capacity_weight'].values[0]),
                            int(v_df[v_df['veh_category'] == 'HD']['payload_capacity_weight'].values[0])]
            num_veh = [f_prob['md_veh'].values[0], f_prob['hd_veh'].values[0]]
    #         print("veh_capacity: ", veh_capacity, " num_veh: ", num_veh)
            fleet_mix = True

        g, payload_nodes, distance, missing_taz = create_graph(df_prob, carr_id, c_prob, depot_loc, fleet_type,taz_df, tt_df)

        if not missing_taz:
            prob_type = add_demand(df_prob, g, payload_nodes)
            prob = VehicleRoutingProblem(g, load_capacity = veh_capacity)
            prob.num_vehicles = num_veh
            prob.time_windows = True
            prob.mixed_fleet = fleet_mix

            if prob_type == 'pickup_delivery':
                prob.pickup_delivery = True
            try:
                print("Problem type: ", prob_type)
                if prob_type == 'pickup_delivery':
                    prob.solve(g, cspy=False, time_limit=60)
                else:
                    prob.solve(time_limit=60)   # Catch an error

                print("Best routes: ", prob.best_routes)
                print("Route departure: ", prob.departure_time)
                print("Route cost: ", prob.best_routes_cost)
                print("Route type: ", prob.best_routes_type )
                print("Route arrival: ", prob.arrival_time)
                print("Route Load: ", prob.best_routes_load)

                # Adding results to the payload dataframe
                if len(df_prob) == 1 and len(prob.best_routes.values()) == 1 :  # If it is a problem with just one payload
                    if prob.best_routes_type[1] == 0:
                        veh_id = vc_df[vc_df['carrier_id']== carr_id]['md_start_id'].values[0]
                    else:
                        veh_id = vc_df[vc_df['carrier_id']== carr_id]['hd_start_id'].values[0]
                        
                    start_time = list(prob.departure_time.values())[0]['Source']
                    print("Adding: ", tour_id, " ", veh_id, " ", start_time, " ", depot_loc)
                    tour_df.loc[tour_id] = [tour_id, veh_id, start_time, depot_loc]
                    carr_filter = (p_df['carrier_id'] == carr_id)
                    sequence_id = 1;
                    p_df.loc[carr_filter,['tour_id']] = tour_id
                    p_df.loc[carr_filter, ['sequence_id']] = sequence_id
                    tour_id +=1
                    if prob_type == 'delivery':
                        delivery_time = list(prob.arrival_time.values())[0][list(payload_nodes.values())[0]]
                        p_df.loc[carr_filter,['del_arrival_time']] = delivery_time
                    elif prob_type == 'pickup':
                        pickup_time = list(prob.arrival_time.values())[0][list(payload_nodes.values())[0]]
                        p_df.loc[carr_filter,['pu_arrival_time']] = pickup_time

            except Exception as e:
                print("Problem did not solve:", e)

INFO:vrpy.vrp:new upper bound : max num stops = 3
INFO:vrpy.vrp:iteration 0, 360.0



 New Problem: Solving problem with heterogeneous fleet for carrier  47
hetegeneous fleet
number of vehicle types =  2
could not find travel time
could not find travel time
Problem type:  pickup


INFO:vrpy.master_solve_pulp:total cost = 360.0
INFO:vrpy.vrp:new upper bound : max num stops = 3
INFO:vrpy.vrp:iteration 0, 360.0


Best routes:  {1: ['Source', 888, 'Sink']}
Route departure:  {1: {'Source': 0, 888: 307.0}}
Route cost:  {1: 360.0}
Route type:  {1: 0}
Route arrival:  {1: {888: 300.0, 'Sink': 307.0}}
Route Load:  {1: 6145}
Adding:  0   25   0   3504

 New Problem: Solving problem with heterogeneous fleet for carrier  382
hetegeneous fleet
number of vehicle types =  2
could not find travel time
could not find travel time
Problem type:  pickup


INFO:vrpy.master_solve_pulp:total cost = 360.0
INFO:vrpy.vrp:new upper bound : max num stops = 3
INFO:vrpy.vrp:iteration 0, 360.0


...

could not find travel time
could not find travel time
Problem type:  pickup_delivery
Problem did not solve: 'int' object is not subscriptable

 New Problem: Solving problem with heterogeneous fleet for carrier  6675552
hetegeneous fleet
number of vehicle types =  2
could not find travel time
could not find travel time
could not find travel time
could not find travel time
could not find travel time


could not find travel time
Problem type:  pickup_delivery
Problem did not solve: 'int' object is not subscriptable

 New Problem: Solving problem with heterogeneous fleet for carrier  6675553
Could not find taz for depot:  2986

 New Problem: Solving problem with heterogeneous fleet for carrier  6675556
hetegeneous fleet
number of vehicle types =  2
could not find travel time
could not find travel time
could not find travel time
could not find travel time
Problem type: 

 pickup_delivery
Problem did not solve: 'int' object is not subscriptable

 New Problem: Solving problem with heterogeneous fleet for carrier  6675568
Could not find taz for zones:  9302  or  435
Could not find taz for depot:  10160

 New Problem: Solving problem with heterogeneous fleet for carrier  7252985
Could not find taz for zones:  8123  or  483
Could not find taz for depot:  4842

 New Problem: Solving problem with heterogeneous fleet for carrier  7252986
Could not find taz for zones:  9128  or  399

 New Problem: Solving problem with heterogeneous fleet for carrier  7252990
Could not find taz for depot:  9249

 New Problem: Solving problem with heterogeneous fleet for carrier  7253120
Could not find taz for zones:  10351  or  310
Could not find taz for depot:  6596

 New Problem: Solving problem with heterogeneous fleet for carrier  7253129
Could not find taz for zones:  7973  or  319
Could not find taz for depot:  9810

 New Problem: Solving problem with heterogeneous fleet f

could not find travel time
could not find travel time
could not find travel time
could not find travel time
could not find travel time
Problem type:  pickup_delivery
Problem did not solve: 'int' object is not subscriptable

 New Problem: Solving problem with heterogeneous fleet for carrier  7253633
hetegeneous fleet
number of vehicle types =  2
could not find travel time
could not find travel time
could not find travel time
could not find travel time
could not find travel time
could not find travel time


could not find travel time
could not find travel time
could not find travel time
could not find travel time
could not find travel time
could not find travel time
Problem type:  pickup_delivery
Problem did not solve: 'int' object is not subscriptable

 New Problem: Solving problem with heterogeneous fleet for carrier  7253634
Could not find taz for zones:  8995  or  416
Could not find taz for depot:  6597

 New Problem: Solving problem with heterogeneous fleet for carrier  7254004
Could not find taz for zones:  464  or  310
Could not find taz for depot:  8465

 New Problem: Solving problem with heterogeneous fleet for carrier  7254075
Could not find taz for zones:  5004  or  366
Could not find taz for depot:  6461

 New Problem: Solving problem with heterogeneous fleet for carrier  7254192
Could not find taz for zones:  5178  or  435

 New Problem: Solving problem with heterogeneous fleet for carrier  7333958
Could not find taz for zones:  7090  or  392
Could not find taz for depot:  52

In [149]:
p_df.head()

,payload_id,carrier_id,commodity,weight,job,pu_zone,del_zone,pu_stop_duration,del_stop_duration,pu_tw_lower,pu_tw_upper,del_tw_lower,del_tw_upper,sequence_id,tour_id,pu_arrival_time,del_arrival_time
0,0,47,1,6145,delivery,0.0,310,0.0,7,0.0,0.0,300,1200,1,0,300,0.0
1,1,382,1,13574,delivery,0.0,316,0.0,18,0.0,0.0,300,1200,1,1,300,0.0
2,2,383,1,6888,delivery,0.0,300,0.0,5,0.0,0.0,300,1200,1,2,300,0.0
3,3,384,1,13574,delivery,0.0,316,0.0,30,0.0,0.0,300,1200,1,3,300,0.0
4,4,386,1,5358,delivery,0.0,316,0.0,14,0.0,0.0,300,1200,1,4,300,0.0


In [150]:
tour_df.head()

,tour_id,veh_id,start_time,start_zone
0,0,25,0,3504
1,1,137,0,4374
2,2,148,0,2569
3,3,165,0,1091
4,4,189,0,3882


In [152]:
filled_df = p_df[p_df['sequence_id'] > 0]

In [153]:
filled_df.head()

,payload_id,carrier_id,commodity,weight,job,pu_zone,del_zone,pu_stop_duration,del_stop_duration,pu_tw_lower,pu_tw_upper,del_tw_lower,del_tw_upper,sequence_id,tour_id,pu_arrival_time,del_arrival_time
0,0,47,1,6145,delivery,0.0,310,0.0,7,0.0,0.0,300,1200,1,0,300,0.0
1,1,382,1,13574,delivery,0.0,316,0.0,18,0.0,0.0,300,1200,1,1,300,0.0
2,2,383,1,6888,delivery,0.0,300,0.0,5,0.0,0.0,300,1200,1,2,300,0.0
3,3,384,1,13574,delivery,0.0,316,0.0,30,0.0,0.0,300,1200,1,3,300,0.0
4,4,386,1,5358,delivery,0.0,316,0.0,14,0.0,0.0,300,1200,1,4,300,0.0


In [163]:
# Save dataframes to files
filled_df.to_csv('input_file/payload_sample.csv', index=False)

In [164]:
tour_df.to_csv('input_file/freight_tours_sample.csv', index=False)